In [ ]:
import pybullet as p
import pybullet_data
import numpy as np
import time
import torch
import SNNModel() from 
%run '/Users/aarushibiswas/Desktop/Brainy_Project/brainy.ipynb'

# add in the SNN model and set it to evaluation mode
snn_model = SNNModel()
snn_model.eval()  

In [ ]:
def initialize_simulation():
    # Start PyBullet simulation
    p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())  # For default assets
    p.setGravity(0, 0, -9.8)
    
    # Load a plane for the ground
    plane_id = p.loadURDF("plane.urdf")
    
    # Load a one-finger gripper
    gripper_id = p.loadURDF("urdf/gripper_finger.urdf", [0, 0, 0.2])
    
    # Define texture objects (e.g., blocks with different friction coefficients)
    textures = []
    for i, friction in enumerate(np.linspace(0.1, 1.2, 12)):
        texture_id = p.loadURDF("cube.urdf", [0, i * 0.2, 0], globalScaling=0.05)
        p.changeDynamics(texture_id, -1, lateralFriction=friction)
        textures.append(texture_id)
    
    return gripper_id, textures

In [ ]:
def apply_force(gripper_id, force):
    # Apply force to the gripper's actuator
    p.setJointMotorControl2(gripper_id, jointIndex=0, controlMode=p.TORQUE_CONTROL, force=force)

def read_tactile_data(gripper_id, texture_id):
    # Simulate barometric and IMU data
    contact_points = p.getContactPoints(gripper_id, texture_id)
    baro_data = sum(point[9] for point in contact_points)  # Normal forces
    imu_data = np.random.uniform(-1, 1, 3)  # Simulated acceleration data
    return baro_data, imu_data

def encode_to_spike_train(baro_data, imu_data):
    # Simple rate encoding for baro and IMU data
    spike_train = np.concatenate([
        np.random.poisson(baro_data, 100),  # Encode baro as spike train
        np.random.poisson(np.abs(imu_data), (3, 100))  # Encode IMU as spike train
    ])
    return torch.tensor(spike_train, dtype=torch.float32)

def decode_snn_output(snn_output):
    # Decode the SNN output to classify texture (softmax probabilities)
    return torch.argmax(snn_output).item()

def adjust_gripper_force(detected_texture):
    # Map textures to target forces
    force_map = np.linspace(5, 20, 12)  # Map texture classes to forces
    return force_map[detected_texture]


In [ ]:
def main():
    gripper_id, textures = initialize_simulation()
    
    for iteration, texture_id in enumerate(textures):
        print(f"Testing texture {iteration + 1}")
        
        # Simulate tactile feedback
        baro_data, imu_data = read_tactile_data(gripper_id, texture_id)
        
        # Encode tactile data into spike trains
        spike_train = encode_to_spike_train(baro_data, imu_data)
        
        # Run the SNN model
        with torch.no_grad():
            snn_output = snn_model(spike_train)
        
        # Decode the output to get the detected texture class
        detected_texture = decode_snn_output(snn_output)
        
        # Adjust the gripper's force based on the detected texture
        target_force = adjust_gripper_force(detected_texture)
        print(f"Applying force: {target_force} N for texture {detected_texture}")
        
        apply_force(gripper_id, target_force)
        
        # Simulate for a short duration
        for _ in range(240):  # Simulate 1 second (240 steps at 240 Hz)
            p.stepSimulation()
            time.sleep(1 / 240.0)
    
    # Disconnect from simulation
    p.disconnect()

if __name__ == "__main__":
    main()
